<a href="https://colab.research.google.com/github/jeppeFQ/BID-M1/blob/main/gymdag_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduktion

## Natural Language Processing (NLP)

Natural Language Processing (**NLP**) er et centralt felt indenfor **AI** (kunstig intelligens). Grundlæggende handler NLP om hvordan en computer kan forstå og fortolke *naturligt sprog*, dvs. menneskeligt talt sprog. Gerne opgaven er at maskiner kan bearbejde dette sprog (og endda kunne producere det).

Med NLP kan maskiner bearbejde tekst og tale på samme måde, som mennesker gør.

> **Spg.:** Hvad vil det sige at et sprog er naturligt?

## Oversættelsesperspektiv

Computeren ikke sprog på samme måde som mennesker gør. De kan læse `1` og de kan læse `0`; men de kan sætte disse tegn sammen i uendelige rækker af varierende kompleksitet. Dvs. mønstre af binære numeriske inputs. NLP handler om at bygge bro mellem den måde, mennesker kommunikerer på, og hvordan maskiner forstår data.

**En IKKE-UDTØMMENDE liste af grundlæggende elementer i oversættelse af naturligt sprog til maskin-læsbart sprog**:

1. ***Tokenization***, som handler om at dele en tekst op i mindre dele, ofte ord eller sætninger. En sætning som "Jeg elsker data!" blive delt op i tre(fire) tokens: ["Jeg", "elsker", "data", "!"].

2. **Stemming og Lemmatization**, som reducerer ord til deres grundform. Fx bliver "løbende" og "løber" reduceret til roden, "løb".

3. **Part-of-Speech Tagging** (POS Tagging), som identificerer ordklasser (som verber, substantiver osv.) for hvert ord i en sætning, hvilket gør det muligt at forstå ordenes funktion i sætningen. [*Ikke aktuelt i dagens workshop*]

4. **Named Entity Recognition** (NER), som identificerer navne på personer, steder eller organisationer i en tekst. For eksempel i sætningen "Aalborg Universitet er et universitet i Danmark" vil "Aalborg Universitet" blive genkendt som en organisation og "Danmark" som et land. [*Ikke aktuelt i dagens workshop*]

Dette oversættelsesperspektiv i en digital kontekst er centralt i dagens workshop.

## Klassifikation i NLP

Et almindeligt "opgave" eller problem i NLP er **klassifikation**. Klassifikation handler om at *tildele et label til en tekst baseret på dens indhold*. I praksis er det en proces, hvor vi anvender en (klassifikations)**algoritme** på et stykke tekst for at **forudsige** den klasse som teksten hører til.

## Superviseret Machine Learning (SML)

I dag er fokus kun på superviseret ML, da vi kun har en enkelt workshop i dag og der vil være for mange statistiske forudsætninger til de to andre hovedtyper.

**SML** fungerer ved, at vi giver `modellen` data, hvor vi kender det rigtige svar. Det kunne være, om en besked er *spam eller ej*, om et produkt er populær baseret på salgsdata, eller hvilken temperatur der vil være i morgen baseret på historiske målinger.

> I besked eksemplet vil det altså sige at vi har et datasæt bestående af SMSer, hvor hver SMS i den data vi træner vores model på er `kodet`, dvs. tilskrevet et `label`, der indikerer om SMSen er spam (`label=1`) eller ikke-spam -- "ham" -- (`label=0`).

Modellen lærer sammenhænge mellem de inputdata (`features`, ord), som vi fodrer den med, og de kendte svar (`labels`, spam/ham). Når modellen er trænet, og den er vurderet til at være god nok, kan vi bruge den til at forudsige `labels` for nye data, hvor vi ikke kender svaret på forhånd.

# Workshop

In [27]:
import pandas as pd
import numpy as np

url = 'https://raw.githubusercontent.com/nijatmammadov/review-classification-nlp/refs/heads/master/IMDB%20Dataset.csv'
data = pd.read_csv(url, sep=",")

print(data.head(10))
len(data)

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
5  Probably my all-time favorite movie, a story o...  positive
6  I sure would like to see a resurrection of a u...  positive
7  This show was an amazing, fresh & innovative i...  negative
8  Encouraged by the positive comments about this...  negative
9  If you like original gut wrenching laughter yo...  positive


50000

Altså,\
Modellens formål er at lære forholdet mellem de inputdata, vi giver den (`features`, "review"), og de kendte `labels` ("sentiment"), så den kan **forudsige** `labels` for *nye, ukendte data*.

Det vil altså sige at vi har med et **klassifikationsproblem** at gøre. Modellen skal forudsige, hvilken kategori noget tilhører, og virke som et spam-filter, hvor vi klassificerer beskeder som enten "spam" eller "ikke-spam", og i en praktisk applikation kan sende indkomne beskeder ind i forskellige mapper, som I kender fra jeres e-mail.

## Modeltræning

I arbejdet med superviseret Machine Learning arbejder vi med vores data som opdelt i hhv. `trænings-` og `testdata`. Den data vi arbejder med, er et datasæt som vi har **kvalitativt kodet** med de korrekte labels ud fra vores forhåndsviden. Med denne opdeling er det muligt både at *træne vores model* og *evaluere vores model*, for at kunne vurdere hvordan modellen performer på nye, usete data.

## **Test**data

Testdata udgør den anden del af den kvalitativt kodede data (her 20%). Testdataene bruges til at evaluere modelens præstation og generaliseringsevne og formålet med testdata er at give et mål for, hvordan modellen vil præstere på nye, usete data. Det vil altså sige at modellen ikke har "set" denne data (og er grunden til at vi skal have Laplace Smoothing...)

## **Opslitning** af data

I Python opslitter vi dataen ved at anvende funktionen `train_test_split` fra `sklearn.model_selection` modulet.

In [29]:
from sklearn.model_selection import train_test_split
# Data er vores DataFrame med anmeldelser og labels
X = data['review']  # Tekst-indholdet (features)
y = data['sentiment']  # Labels (targets)

In [30]:
print(X)

0        One of the other reviewers has mentioned that ...
1        A wonderful little production. <br /><br />The...
2        I thought this was a wonderful way to spend ti...
3        Basically there's a family where a little boy ...
4        Petter Mattei's "Love in the Time of Money" is...
                               ...                        
49995    I thought this movie did a down right good job...
49996    Bad plot, bad dialogue, bad acting, idiotic di...
49997    I am a Catholic taught in parochial elementary...
49998    I'm going to have to disagree with the previou...
49999    No one expects the Star Trek movies to be high...
Name: review, Length: 50000, dtype: object


In [ ]:
# Split data i trænings- og test-sæt.
# Random state er et "seed", der bestemmer det tilfældige udtræk.
# Hvis i har samme random state som her, skulle I få samme resultat.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [31]:
print(X_test)

11872    This movie was beyond awful, it was a pimple o...
40828    As of this writing John Carpenter's 'Halloween...
36400    I must admit a slight disappointment with this...
5166     Oh dear! The BBC is not about to be knocked of...
30273    its a totally average film with a few semi-alr...
                               ...                        
5703     I liked this movie. Unlike other thrillers you...
36992    I've seen this movie more than once. It was on...
14005    There have been many movies about people retur...
29455    Heftig og Begeistret (Intense and Enthusiastic...
36904    This is probably Karisma at her best, apart fr...
Name: review, Length: 10000, dtype: object


In [32]:
print(y_test)

11872    negative
40828    positive
36400    positive
5166     negative
30273    negative
           ...   
5703     positive
36992    positive
14005    positive
29455    negative
36904    positive
Name: sentiment, Length: 10000, dtype: object


# Naive Bayes klassifikation i praksis

Repitation af formål og hvad vi vil implementere i Python er:  

> Sandsynligheden for at en anmeldelse er positiv, baseret på fremkomsten/tilstedeværelsen af et givent ord, er proportionelt til sandsynligheden for at ordet fremkommer i positive anmeldelser og den *a priori* sandsynlighed for at en tilfældig anmeldelse er positiv

$$
P(\text{positiv}|ord)\propto P(ord|\text{positiv}) P(\text{positiv})
$$

> **Spg.:** Hvordan implimenterer vi denne model i Python på en måde, der kan "lære" maskinen at genkende spam-SMSer?

## 1. Indlæs ML/AI biblioteker



In [76]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

## 2. Bag-of-Word vektorisering

In [74]:
# Liste af stopord
yderlige_stop_words = {"movie", "film", "like", "br", "story", "people"}
stop_words = list(ENGLISH_STOP_WORDS.union(yderlige_stop_words))

print(stop_words)

['nobody', 'hereupon', 'made', 'former', 'much', 'his', 'thus', 'six', 'ever', 'elsewhere', 'because', 'nevertheless', 'what', 'inc', 'thru', 'four', 'call', 'is', 'too', 'whereas', 'twelve', 'together', 'then', 'anyway', 'behind', 'such', 'give', 'neither', 'during', 'should', 'con', 'thereafter', 'on', 'story', 'fire', 'never', 'how', 'ltd', 'latterly', 'he', 'several', 'full', 'throughout', 'become', 'almost', 'mine', 'except', 'meanwhile', 'same', 'everywhere', 'here', 'de', 'people', 'put', 'somewhere', 'that', 'we', 'however', 'seems', 'via', 'ourselves', 'for', 'yourselves', 'her', 'became', 'of', 'hers', 'own', 'un', 'first', 'whose', 'whence', 'hundred', 'nowhere', 'the', 'with', 'co', 'so', 'itself', 'describe', 'take', 'fifty', 'hereby', 'among', 'nor', 'from', 'within', 'might', 'go', 'but', 'me', 'something', 'etc', 'who', 'why', 'hence', 'off', 'as', 'namely', 'fifteen', 'between', 'beside', 'about', 'else', 'thereby', 'whereupon', 'serious', 'none', 'found', 'well', 'nam

In [ ]:
# Initialisering af CountVectorizer-klassen med stopord og filtre for ikke- og særligt fremkomne ord
vectorizer_bow = CountVectorizer(stop_words=stop_words, min_df=1, max_df=0.75)

In [60]:
# CountVectorizer er en klasse i scikit-learn.
# Når vi kører vectorizer_bow = CountVectorizer() initialicerer vi klasse
# med basis "indstillingerne" relateret til klasse (i.e., indbyggede metoder og datastruktur)
# Eller de specikke indstillinger vi giver som indput (max_df=0.75, stop_words='english'). min_df=1 er default.
# Derfor får vi bare:
print(vectorizer_bow)

CountVectorizer(max_df=0.75,
                stop_words=['nobody', 'hereupon', 'made', 'former', 'much',
                            'his', 'thus', 'six', 'ever', 'elsewhere',
                            'because', 'nevertheless', 'what', 'inc', 'thru',
                            'four', 'call', 'is', 'too', 'whereas', 'twelve',
                            'together', 'then', 'anyway', 'behind', 'such',
                            'give', 'neither', 'during', 'should', ...])


In [61]:
# Transformer træningsdata (Splittet fra X) til BoW
X_train_bow = vectorizer_bow.fit_transform(X_train)

`.fit_transform()` er en dedikeret metode i `CountVectorizer` klassen:

1. `fit_transform` er "indkapslet" i `CountVectorizer` klassen.

2. Vi behøves derfor ikke vide detaljerne i hvordan metoden vektoriserer vores data givet input, da dette er abstrakseret i det simple kald: `klasse.metode(input)`

3. `fit_transform` "tilhører" derfor også klassen og opererer kun på data, vi associerer med denne klasse. Vi gør det med `associeret data` = `klasse.metode(input)`.
  1. `fit`: er en metode til at definere den interne data, givet som vektor af unikke ord og deres frekvenser.
  2. `transform`: transform er en metode, der transformere data indputtet til den interne datastruktur som numeriske repræsentationer for hvert ord (en matrice med ordfrekvenser)

In [62]:
print(X_train_bow)

  (0, 58078)	1
  (0, 77134)	1
  (0, 24220)	2
  (0, 45630)	3
  (0, 17900)	1
  (0, 34157)	1
  (0, 41406)	1
  (0, 79035)	1
  (0, 48120)	1
  (0, 71446)	1
  (0, 79835)	2
  (0, 89116)	1
  (0, 78923)	1
  (0, 31307)	1
  (0, 12090)	1
  (0, 91984)	1
  (0, 77225)	3
  (0, 32945)	3
  (0, 14991)	1
  (0, 90459)	2
  (0, 80783)	1
  (0, 91264)	1
  (0, 28500)	2
  (0, 90470)	1
  (0, 33794)	3
  :	:
  (39999, 22614)	1
  (39999, 27820)	1
  (39999, 57684)	1
  (39999, 25078)	2
  (39999, 1854)	1
  (39999, 66494)	1
  (39999, 6809)	1
  (39999, 34297)	1
  (39999, 72832)	1
  (39999, 34190)	1
  (39999, 64164)	1
  (39999, 41330)	1
  (39999, 46678)	1
  (39999, 64159)	1
  (39999, 40937)	1
  (39999, 25135)	1
  (39999, 32631)	1
  (39999, 5056)	1
  (39999, 20995)	1
  (39999, 35079)	1
  (39999, 15487)	1
  (39999, 71594)	1
  (39999, 37618)	1
  (39999, 46510)	2
  (39999, 15997)	1


In [63]:
# Transformer testdata
# I testdataen bruger vi den allerede definerede ordliste fra træningsdataene.
# og behøves derfor ikke også at kalde fit.
X_test_bow = vectorizer_bow.transform(X_test)

In [64]:
print(X_test_bow)

  (0, 38)	1
  (0, 1584)	1
  (0, 1881)	1
  (0, 4805)	1
  (0, 6427)	1
  (0, 6548)	1
  (0, 8871)	1
  (0, 9073)	1
  (0, 10287)	1
  (0, 11874)	1
  (0, 13700)	1
  (0, 17847)	1
  (0, 19215)	1
  (0, 25135)	1
  (0, 25306)	1
  (0, 27421)	1
  (0, 30404)	1
  (0, 30926)	1
  (0, 33755)	1
  (0, 34904)	1
  (0, 36744)	1
  (0, 37472)	1
  (0, 37476)	1
  (0, 37796)	1
  (0, 38478)	1
  :	:
  (9999, 47047)	1
  (9999, 47155)	1
  (9999, 53942)	1
  (9999, 54557)	1
  (9999, 55613)	1
  (9999, 59131)	1
  (9999, 60413)	1
  (9999, 61129)	1
  (9999, 61901)	1
  (9999, 63268)	1
  (9999, 64031)	1
  (9999, 64241)	1
  (9999, 66658)	1
  (9999, 66692)	2
  (9999, 73053)	1
  (9999, 73260)	1
  (9999, 73764)	1
  (9999, 78041)	1
  (9999, 79771)	1
  (9999, 82479)	1
  (9999, 82705)	1
  (9999, 84566)	1
  (9999, 87634)	1
  (9999, 89299)	1
  (9999, 92475)	1


## Modellering af data

Med samme logik initialiserer vi her et Naive Bayes objekt, til at indkapse data i den rette struktur og modellerede sandsynligheder fra træning og forudsigelse. Vi organiserer altså de to analytiske dimensioner: træning og test.

Hermed er den ellers meget komplekse kode derfor abstrakseret, således at vi har en slagt "interface" med nogle dedikerede og relevante metoder (her fx `fit` og `predict`).

Det gør det derfor ligefremt at udvide eller ændre vores model og analyse, idet vi kan bruge de identistiske metoder for andre algoritmer end kun Naive Bayes, uden det kræver nye former for implementering af kode (nedarvning og foranderlighed).

In [65]:
# Naive Bayes-model
model_bow = MultinomialNB()
print(model_bow)

MultinomialNB()


In [66]:
# Træn modellen og tilskriv de trænede sandsynligheder til objektet "model_bow"
model_bow.fit(X_train_bow, y_train)

MultinomialNB()

In [67]:
# Alt det følgende behøves i ikke at forstå. Det er udelukkende for at printe
# sandsynligheder for de enkelte ord fra vores NB model.

## Udtræk sandsynlighederne fra modeller
feature_names = vectorizer_bow.get_feature_names_out()

log_prob = model_bow.feature_log_prob_

probabilities = np.exp(log_prob)

df_probs = pd.DataFrame(probabilities.T, columns=model_bow.classes_, index=feature_names)

## Top-10 prd
n_top = 10

## Loop gennem hver klasse
for class_label in df_probs.columns:
    print(f"\nKlasse: {class_label}")

    # Sort words by probability for the current class
    sorted_probs = df_probs[class_label].sort_values(ascending=False)

    # Get top N words
    top_words = sorted_probs.head(n_top)
    print("\nOrd med de højeste sandsynligheder for klassen:")
    for word, prob in top_words.items():
        print(f"{word}: {prob:.4f}")



Klasse: negative

Ord med de højeste sandsynligheder for klassen:
just: 0.0080
bad: 0.0057
good: 0.0056
time: 0.0048
really: 0.0048
don: 0.0041
make: 0.0036
movies: 0.0032
plot: 0.0032
acting: 0.0031

Klasse: positive

Ord med de højeste sandsynligheder for klassen:
good: 0.0055
just: 0.0052
great: 0.0048
time: 0.0047
really: 0.0039
love: 0.0032
best: 0.0031
life: 0.0030
way: 0.0029
films: 0.0028


> Kunne man tænke at mere tid på databehandling ville være nyttig?

In [68]:
# Forudsig på testdata og tilskriv de estimerede sandsynligheder til "model_bow" objektet
y_pred_bow = model_bow.predict(X_test_bow)
print(y_pred_bow)

['negative' 'positive' 'positive' ... 'positive' 'positive' 'positive']


## Opsummeret

1. Vi initialiserer et objekt (instance) ved at kalde `model_bow = MultinomialNB()`. Et objekt med dets egen interne data- og inputstruktur (i.e., numerisk repræsentation af data og modellerede sandsynligheder) og dedikerede metoder til at bearbejde og tilgå disse informationer.
**Vi interagerer med objektet og dets data vha. objektets indbyggede metoder:**
2. Med den indbyggede `fit`-metode kan vi træne en NB model på den tilskrevne data og *opdatere objektets interne tilstand*.
3. Med den indbyggede `predict`-metode kan vi udregne forudsagte sandsynligheder og klasser og gemme/tilskrive denne data til objektet og igen  *opdaterer vi objektets interne tilstand*.     
**Det centrale er at alle detaljer om algoritme forbliver "skujlt" for brugeren, da den er indbygget i objektet og initialiseres vha. de indbyggede metoder i en slags interface.**


## Test og evaluering

In [69]:
results = pd.DataFrame({
    "Korrekt Label": y_test,
    "Forudsagt Label": y_pred_bow
})

print(results)

      Korrekt Label Forudsagt Label
11872      negative        negative
40828      positive        positive
36400      positive        positive
5166       negative        positive
30273      negative        negative
...             ...             ...
5703       positive        positive
36992      positive        negative
14005      positive        positive
29455      negative        positive
36904      positive        positive

[10000 rows x 2 columns]


In [70]:
# Evaluering af modellen
# Vi tager den oprindelige data med de "ægte" labels (y_test) og den data vi konstruerer (y_pred_bow)
# med vores trænede model gemt i objektet (model_bow).
# Disse to vektorer af værdier (rigtigt og forudsagt label) og udregner evalueringsmål med funktionen classification_report()
print("Naive Bayes med BoW pre-processing af naturligt sprog\n\n")
print(classification_report(y_test, y_pred_bow))

Naive Bayes med BoW pre-processing af naturligt sprog


              precision    recall  f1-score   support

    negative       0.85      0.87      0.86      4979
    positive       0.87      0.84      0.86      5021

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000



# Hvad kan i gøre herfra (sammen med elever)

- https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/L4OAKN

- https://manifesto-project.wzb.eu/datasets/MPDS2024a

- https://github.com/erikgahner/PolData

### 1. Håndkodning af ny data (scrapet eller downloadet)

### 2. Øv med allerede eksisterende data


# **Spam-filter**

## Konkret øvelse til at gå i gang med:

> Det eneste, der skal ændres ift. koden ovenfor er at i koden nedenfor hedder kolonnerne ikke "review" og "sentiment". Brug `print(data)` for at se hvad variablene hedder i den nye data

```
X = data['review']  # Tekst-indholdet (features)
y = data['sentiment']  # Labels (targets)
```

Derudover skal de "yderlige stopord" sandsynligvis også ændres. Eller ej?

```
# Liste af stopord
yderlige_stop_words = {"movie", "film", "like", "br", "story", "people"}
stop_words = list(ENGLISH_STOP_WORDS.union(yderlige_stop_words))
```






In [25]:
import pandas as pd

url = 'https://raw.githubusercontent.com/manjit-baishya-datascience/Spam-Email-Detection/refs/heads/main/email_data.csv'
data = pd.read_csv(url, sep=",")

print(data.head(10)) # HINT: navnene er "Category", der skal være `y` og "Message" som skal være `X`
len(data)

  Category                                            Message
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...
5     spam  FreeMsg Hey there darling it's been 3 week's n...
6      ham  Even my brother is not like to speak with me. ...
7      ham  As per your request 'Melle Melle (Oru Minnamin...
8     spam  WINNER!! As a valued network customer you have...
9     spam  Had your mobile 11 months or more? U R entitle...


5572

# Dansk tekst

Skal I have danske stopord er processen:


In [77]:
!pip install nltk

In [78]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer

# Download stopord
nltk.download('stopwords')
from nltk.corpus import stopwords

# Danske stopord liste
danske_stop_ord = stopwords.words('danish')

# Yderlige stopord, hvis relevant
stop_words = danske_stop_ord + ["customword1", "customword2"]

print(stop_words)

['og', 'i', 'jeg', 'det', 'at', 'en', 'den', 'til', 'er', 'som', 'på', 'de', 'med', 'han', 'af', 'for', 'ikke', 'der', 'var', 'mig', 'sig', 'men', 'et', 'har', 'om', 'vi', 'min', 'havde', 'ham', 'hun', 'nu', 'over', 'da', 'fra', 'du', 'ud', 'sin', 'dem', 'os', 'op', 'man', 'hans', 'hvor', 'eller', 'hvad', 'skal', 'selv', 'her', 'alle', 'vil', 'blev', 'kunne', 'ind', 'når', 'være', 'dog', 'noget', 'ville', 'jo', 'deres', 'efter', 'ned', 'skulle', 'denne', 'end', 'dette', 'mit', 'også', 'under', 'have', 'dig', 'anden', 'hende', 'mine', 'alt', 'meget', 'sit', 'sine', 'vor', 'mod', 'disse', 'hvis', 'din', 'nogle', 'hos', 'blive', 'mange', 'ad', 'bliver', 'hendes', 'været', 'thi', 'jer', 'sådan', 'customword1', 'customword2']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
